In [5]:
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from fake_useragent import UserAgent
import time
import pandas as pd
import chardet
import threading
import pandas as pd
import math
import os

file_name = 'awin200.xlsx'


# data
raw = pd.read_excel(file_name)
raw['Status'] = ''
raw['Contact Name'] = ''
raw['Contact Email'] = ''
raw['Contact number'] = ''
raw['Attempted'] = ''
# Number of threads/drivers to use
num_threads = 10

# Divide data into equal parts
data_per_thread = len(raw) // num_threads

# Function to perform scraping for a specific subset of data
def scrape_data(start_index, end_index):
    driver = webdriver.Chrome()
    wait = WebDriverWait(driver, 10)

    # Create a unique output file name based on the thread's name
    output_file_path = f"output_file_thread_{threading.current_thread().name}.xlsx"

    def login():
        retry=3
        while retry > 0:
            # navigate to the website (removed this line as we don't need to navigate again)
            driver.get("https://ui.awin.com/dashboard/awin/publisher/1363573/gb")

            try:
                # enter the email and password
                email_field = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="email"]')))
                email_field.send_keys("demand@joingekko.com")
                password_field = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="password"]')))
                password_field.send_keys("DubaiSunset2023")
                # click the login button
                login_button = driver.find_element(By.XPATH, '//*[@id="login"]/span[1]')
                login_button.click()
                break
            except:
                retry-=1
                pass

    login()

    for i in range(start_index, end_index):
        try:
            xyz = str(int(raw['Program ID'].iloc[i]))
            driver.get(f"https://ui.awin.com/awin/affiliate/1363573/merchant-profile/{xyz}")
            try:
                time.sleep(3)
                try:
                    contact_email = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="descriptionLongContentView"]/p[contains(text(), "@")]')))
                    raw.iloc[i, raw.columns.get_loc('Contact Email')] = contact_email.text
                except:
                    pass        
                try:
                    contact_email = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="descriptionLongContentView"]/p[10]/a')))
                    raw.iloc[i, raw.columns.get_loc('Contact Email')] = contact_email.text
                except:
                    pass  
                try:
                    contact_name = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="content"]/div[1]/div[2]/div/div/div/ul/li/h4')))
                except:
                    pass
                try:
                    contact_email = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="content"]/div[1]/div[2]/div/div/div/ul/li/table/tbody/tr[1]/td[2]/a')))
                except:
                    pass
                try:
                    contact_number = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="content"]/div[1]/div[2]/div/div/div/ul/li/table/tbody/tr[2]/td[2]')))
                except:
                    pass
                try:
                    s = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="actionProgramme"]')))
                except:
                    pass
                # results
                try:
                    raw.iloc[i, raw.columns.get_loc('Contact Name')] = contact_name.text
                except:
                    pass
                try:
                    raw.iloc[i, raw.columns.get_loc('Contact Email')] = contact_email.text
                except:
                    pass
                try:
                    raw.iloc[i, raw.columns.get_loc('Contact number')] = contact_number.text
                except:
                    pass
                try:
                    raw.iloc[i, raw.columns.get_loc('Status')] = s.text 
                except:
                    pass    
            except:
                pass
            try:
                # Click on "Join Program" or perform desired action
                join_link = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="actionProgramme"]/p')))
                join_link.click()
                time.sleep(6)
                # Find the message field and enter the advertiser message
                message_field = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="message"]')))
                message='JoinGekko.com is a sub-network that works with over 2,000+ media partners worldwide to bring your affiliate sales to a new height! Our platform streamlines partnerships, leveraging commerce content for sustainable revenue. With Gekko, recruitment and management become effortless, connecting you with content, creators, and influencers. We ensure transparent monitoring and compliance for optimal brand performance. If you have any questions, do not hesitate to contact kenny.huang@joingekko.com'
                message_field.send_keys(message)
                checkbox = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="accepted"]')))
                checkbox.click()   
                # Find and click on the submit program
                join_button = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="membershipModal"]/div[2]/button[2]')))
                join_button.click()
                # OK 
                OK = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="popup_ok"]')))
                OK.click()
                #status
            
                #raw.iloc[i, raw.columns.get_loc('Status')] = 'Pending'
                try:
                    s = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="actionProgramme"]')))
                except:
                    pass
                try:
                    raw.iloc[i, raw.columns.get_loc('Status')] = s.text 
                except:
                    pass  
                raw.iloc[i, raw.columns.get_loc('Attempted')] = 'Yes'
            except:
                pass     
        except:
            login()

        try:
            # Save the output_file for each subset_data
            raw[start_index:end_index].to_excel(output_file_path, index=False)
        except:
            print('close output file')

    driver.quit()

# Create and start threads with their respective data subsets
threads = []
for i in range(num_threads):
    start_index = i * data_per_thread
    end_index = start_index + data_per_thread
    if i == num_threads - 1:
        # For the last thread, include any remaining data to cover cases where len(raw) % num_threads != 0
        end_index = len(raw)
    thread = threading.Thread(target=scrape_data, args=(start_index, end_index))
    thread.start()
    threads.append(thread)

# Wait for all threads to finish
for thread in threads:
    thread.join()

print("All threads have finished.")

def aggregate_excel_parts(input_prefix, output_file):
    # Initialize an empty DataFrame to store the aggregated data
    aggregated_df = pd.DataFrame()

    # Find all the divided Excel files with the given prefix
    divided_files = [file for file in os.listdir() if file.startswith(input_prefix)]

    # Read each divided Excel file and concatenate its data to the aggregated DataFrame
    for file in divided_files:
        df = pd.read_excel(file)
        aggregated_df = pd.concat([aggregated_df, df])

    # Save the aggregated DataFrame to a new Excel file
    aggregated_df.to_excel(output_file, index=False)
    print(f"Aggregated data saved to {output_file}")
    
if __name__ == "__main__": 
    
    input_prefix = 'output_file_thread_Thread'
    output_file = 'aggregated_output_file.xlsx'
    aggregate_excel_parts(input_prefix, output_file)

All threads have finished.
Aggregated data saved to aggregated_output_file.xlsx
